In [2]:
from google.datalab import Context
import google.datalab.bigquery as bq
import google.datalab.storage as storage
import pandas as pd
import numpy as np
try:
  from StringIO import StringIO
except ImportError:
  from io import BytesIO as StringIO
import matplotlib.pyplot as plt
%matplotlib inline

Please consider only 2007 & 2008 from stats to solve below questions:
* Which Carrier performs better?
* When is the best time of day/day of week/time of year to fly to minimise delays?
* Do older planes suffer more delays?
* Can you detect cascading failures as delays in one airport create delays in others? Are there critical links in the system?
* Create a model to predict flight delays?
* How well does weather predict plane delays?

### Below code is to extract files from internet and unzip them

In [ ]:
import requests,bz2,os
def pull_decompress(year):
  '''
  This function downloads file to local drive and decompresses the file. This has function takes argument as the year from 1988 to 2008
  '''
  try:
    url= "http://stat-computing.org/dataexpo/2009/"+str(year)+".csv.bz2"
    r =requests.get(url)
    try:
      os.remove(str(year)+'.csv.bz2')
      os.remove(str(year)+'.csv')
    except OSError:
      pass
    with open(str(year)+'.csv.bz2','wb') as f:
      f.write(r.content)
    data=bz2.BZ2File(str(year)+'.csv.bz2','rb').read()
    open(str(year)+'.csv','wb').write(data)
    return True
  except IOError:
    return False
    

In [ ]:
schema=bq.Schema.from_data(df2)
tbl_2008=bq.Table('Airplandata.tbl_data').create(schema=schema,overwrite=True)

In [ ]:
tbl_2008.load('gs://srimldatapoint/20081.csv',mode='append',source_format='csv',csv_options=bq.CSVOptions(skip_leading_rows = 1))

In [ ]:
tbl_2008.load('gs://srimldatapoint/2007.csv',mode='append',source_format='csv',csv_options=bq.CSVOptions(skip_leading_rows = 1))

In [ ]:
!gsutil cp '2008.csv' gs://srimldatapoint/20081.csv

In [ ]:
!gsutil cp '2007.csv' gs://srimldatapoint/2007.csv

In [ ]:
df2[((df2['DepTime']=='')| (df2['ArrTime']==''))&(df2['DepDelay']!='')][['Year','DepTime','ArrTime','DepDelay']].shape
df3[df3['DepTime']==''].head(10)
df3['DepTime']=df3['DepTime']*1

In [ ]:
df.isnull().sum()/df.shape[0]

In [ ]:
df.head(10)

In [ ]:
%%gcs read --object gs://srimlbucket/20081.csv --variable data_2008

In [ ]:
df = pd.read_csv(StringIO(data_2008))
schema = bq.Schema.from_data(df)

# Create the dataset
bq.Dataset('Airplandata').create()

# Create the table
sample_table = bq.Table('Airplandata.2008').create(schema = schema, overwrite = True)

In [ ]:
sample_table.load('gs://srimlbucket/20081.csv', mode='append',
                  source_format = 'csv', csv_options=bq.CSVOptions(skip_leading_rows = 1))

In [ ]:
print(data_2008)

### Downloading Data from  sources using bash commands

In [ ]:
!wget "http://stat-computing.org/dataexpo/2009/airports.csv"
!wget "http://stat-computing.org/dataexpo/2009/carriers.csv"
!wget "http://stat-computing.org/dataexpo/2009/plane-data.csv"
!wget "http://stat-computing.org/dataexpo/2009/2008.csv.bz2"
!bzip2 -d '2008.csv.bz2'
!wget "http://stat-computing.org/dataexpo/2009/2007.csv.bz2"
!bzip2 -d '2007.csv.bz2'

### Loading data from local drive to dataframe

In [3]:
df_2008=pd.read_csv('2008.csv')
df_2007=pd.read_csv('2007.csv')
airport_df=pd.read_csv('airports.csv')
carrier_df=pd.read_csv('carriers.csv')
planes_df=pd.read_csv('plane-data.csv')
main_df=pd.concat([df_2008,df_2007],ignore_index=True)

In [ ]:
print(df_2008.shape,df_2007.shape)

In [44]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14462943 entries, 0 to 14462942
Data columns (total 29 columns):
Year                 int64
Month                int64
DayofMonth           int64
DayOfWeek            int64
DepTime              float64
CRSDepTime           int64
ArrTime              float64
CRSArrTime           int64
UniqueCarrier        object
FlightNum            int64
TailNum              object
ActualElapsedTime    float64
CRSElapsedTime       float64
AirTime              float64
ArrDelay             float64
DepDelay             float64
Origin               object
Dest                 object
Distance             int64
TaxiIn               float64
TaxiOut              float64
Cancelled            int64
CancellationCode     object
Diverted             int64
CarrierDelay         float64
WeatherDelay         float64
NASDelay             float64
SecurityDelay        float64
LateAircraftDelay    float64
dtypes: float64(14), int64(10), object(5)
memory usage: 3.1+ GB


In [ ]:
airport_df.head(10)

In [ ]:
carrier_df.head(10)
carrier_df[carrier_df['Code']=='F9']

In [ ]:
pd.options.display.float_format = '{:.04f}'.format
imp_col=['Cancelled','ArrDelay','DepDelay','AirTime','Distance']
main_df[imp_col].describe()
#temp=main_df.styles.format()

In [ ]:
main_df[main_df['AirTime']==0].groupby('UniqueCarrier')['ArrTime'].count()

In [ ]:
main_df[main_df['Cancelled']==1].groupby('UniqueCarrier')['UniqueCarrier'].count()

In [ ]:
main_df[main_df['AirTime']==0][['UniqueCarrier','DepTime','CRSDepTime','ArrTime','AirTime']]

In [ ]:
#F9 has lowest percentage of Cancellations compages to all the Carriers
#Total cancelled flights are 2.061
#main_df[main_df['Cancelled']==1].shape[0]/main_df.shape[0]
Cancel_series=pd.Series(main_df[main_df['Cancelled']==1]['UniqueCarrier'].value_counts()/main_df['UniqueCarrier'].value_counts()).sort_values()

In [ ]:
Cancel_series

In [ ]:
Non_cacel=main_df[main_df['Cancelled']==0]

In [ ]:
Non_cacel.shape,main_df.shape

In [ ]:
fig=plt.figure()
ax=fig.add_subplot(111)
Cancel_series.plot(kind='bar',ax=ax)
ax.set_xlabel('Unique Carrier')

In [ ]:
Non_cacel[['UniqueCarrier','ArrDelay','DepDelay']].groupby(['UniqueCarrier']).sum().sort_values(by='DepDelay').plot(kind='bar')

In [ ]:
pd.Series(Non_cacel['UniqueCarrier'].value_counts()/Non_cacel.shape[0]).plot(kind='bar')

In [ ]:
Non_cacel['Origin'].unique().shape[0], Non_cacel['Dest'].unique().shape[0]

In [ ]:
Non_cacel['Origin'].value_counts().sort_values(ascending=False)[0:20]

In [ ]:
Non_cacel['Dest'].value_counts().sort_values(ascending=False)[0:20]

In [ ]:
def group_car(main_df):
  cancel_df=main_df[main_df['Cancelled']==1]['UniqueCarrier'].value_counts()/main_df['UniqueCarrier'].value_counts()
  non_cancel=main_df[main_df['Cancelled']==0]
  delay_dept=non_cancel[(non_cancel['DepDelay']!=0) |(non_cancel['ArrDelay']!=0)].groupby('UniqueCarrier')['DepDelay'].count()/non_cancel['UniqueCarrier'].value_counts()
  share_flight=non_cancel['UniqueCarrier'].value_counts()/non_cancel.shape[0]
  non_cancel['squared_val_Dep_delay']= non_cancel['DepDelay']**2
  non_cancel['squared_val_Arr_delay']=non_cancel['ArrDelay']**2
  total_delay=np.sqrt(non_cancel.groupby('UniqueCarrier')['squared_val_Dep_delay'].sum()/non_cancel['UniqueCarrier'].value_counts())+np.sqrt(non_cancel.groupby('UniqueCarrier')['squared_val_Arr_delay'].sum()/non_cancel['UniqueCarrier'].value_counts())
  final_del_data=pd.concat([share_flight,cancel_df,delay_dept,total_delay],axis=1,ignore_index=False)
  final_del_data.columns=['flight_share','Cacel_per','dep_del_per','total_delay']
  #Arr_delay=np.sqrt(non_cancel.groupby('UniqueCarrier')['squared_val_Arr_delay'].sum()/non_cancel['UniqueCarrier'].value_counts())
  return final_del_data

def perform_eval(df):
  ''' This function should contain a calculative model which can measure performance of carries
  based on number of cancellation per year, 
  number of delayed departure,
  percentage of flights its making, absolute number of total minutes of delayed departure
  f(x)= 0.35*per_cancell+0.25*delay_departure+0.25*share_of_flights+0.15*squared_val_delayed
  '''
  df['Non_Perform_index']=0.35*df['Cacel_per']+0.15*df['dep_del_per']+0.15*(1/df['flight_share'])+0.35*df['total_delay']
  #df['Perform_index']=0.35*df['Cacel_per']+0.3*df['dep_del_per']+0.35*df['total_delay']
  global carrier_df
  if 'Code' in carrier_df.columns:
    carrier_df.set_index('Code',inplace=True)
  df=df.merge(carrier_df,how='left',left_index=True,right_index=True)
  df= df.sort_values(by='Non_Perform_index',ascending=True)
  return df
def HighPerform_carrier(df):
  '''The input for this function is from perform_eval'''
  return(df['Description'][0],df['Non_Perform_index'][0])

def Perform_full(main_df):
  return HighPerform_carrier(perform_eval(group_car(main_df)))

In [ ]:
#data_final= group_car(main_df)
#data_final_eval=perform_eval(data_final)
#This line of code provides High performing carrier
name,index=Perform_full(main_df)
print("The High performance Carrier is {} and its score of index is {:.2f}".format(name,index))

In [ ]:
df_2008['Date']=pd.to_datetime(pd.DataFrame({'year':df_2008['Year'],'month':df_2008['Month'],'day':df_2008['DayofMonth']}))

In [ ]:
#df_2008[0:100].to_csv('Export_csv.csv')
df_2008.head(10)

In [ ]:
#df_2008_1=df_2008.set_index('Date')
f = lambda x: np.max(x)-np.min(x)
def x_bar(x):
  return np.mean(x)/np.size(x)
def x_mean(x):
  return np.max(x)-np.min(x)
def x_Ucl(x):
  return np.mean(x)/np.size(x)+((np.max(x)-np.min(x))*3/(3.931*np.sqrt(np.size(x))))
def x_Lcl(x):
  return np.mean(x)/np.size(x)-((np.max(x)-np.min(x))*3/(3.931*np.sqrt(np.size(x))))
def x_test(x):
  ucl=x_Ucl(x)
  lcl=x_Lcl(x)
  return np.sum((x>ucl)|(x<lcl))/np.size(x)

non_cancel=main_df[main_df['Cancelled']==0]
non_cancel['Date']= pd.to_datetime(pd.DataFrame({'year':non_cancel['Year'],'month':non_cancel['Month'],'day':non_cancel['DayofMonth']}))
sample_data=non_cancel[['Date','UniqueCarrier','DepDelay','ArrDelay']].groupby(['Date','UniqueCarrier']).apply(lambda x: x.sample(n=25,random_state=100,replace=False).mean()).reset_index()
carrier_sel=sample_data[['UniqueCarrier','DepDelay','ArrDelay']].groupby('UniqueCarrier').agg([x_bar,x_Lcl,x_Ucl,x_test])
carrier_sel.columns = ['_'.join(col) for col in carrier_sel.columns]
carrier_sel['Final_test']= (carrier_sel['DepDelay_x_test']+carrier_sel['ArrDelay_x_test'])/2
carrier_sel=carrier_sel.sort_values(by='Final_test',ascending=False)
if 'Code' in carrier_df.columns:
    carrier_df.set_index('Code',inplace=True)
carrier_sel=carrier_sel.merge(carrier_df,how='left',left_index=True,right_index=True)
print("Best carrier with minimal delays is {} and worst carrier with major delay is {}".format(carrier_sel['Description'][0],carrier_sel['Description'][-1]))
#non_cancel_avg.head(10)

In [ ]:
sample_data=non_cancel[['Date','DayOfWeek','DepDelay','ArrDelay']].groupby(['Date','DayOfWeek']).apply(lambda x: x.sample(n=25,random_state=100,replace=False).mean()).reset_index()
#sample_data= non_cancel_avg.groupby('DayOfWeek')
#sample_data.head(10)
sample_data_Dow=sample_data[['DayOfWeek','DepDelay','ArrDelay']].groupby('DayOfWeek').agg([x_bar,x_Lcl,x_Ucl,x_test])
sample_data_Dow.columns = ['_'.join(col) for col in sample_data_Dow.columns]
sample_data_Dow['Final_test']= (sample_data_Dow['DepDelay_x_test']+sample_data_Dow['ArrDelay_x_test'])/2
sample_data_Dow=sample_data_Dow.sort_values(by='Final_test',ascending=True)
print("Best time to minimize delays is to travel on {} day of week".format(sample_data_Dow.index[0]))

In [ ]:
sample_data=non_cancel[['Date','Month','DepDelay','ArrDelay']].groupby(['Date']).apply(lambda x: x.sample(n=25,random_state=100,replace=False).mean()).reset_index()
#sample_data= non_cancel_avg.groupby('Month')
#sample_data.head(10)
sample_data_Mnth=sample_data[['Month','DepDelay','ArrDelay']].groupby('Month').agg([x_bar,x_Lcl,x_Ucl,x_test])
sample_data_Mnth.columns = ['_'.join(col) for col in sample_data_Mnth.columns]
sample_data_Mnth['Final_test']= (sample_data_Mnth['DepDelay_x_test']+sample_data_Mnth['ArrDelay_x_test'])/2
sample_data_Mnth=sample_data_Mnth.sort_values(by='Final_test',ascending=True)
print("Best time to minimize delays is to travel in {} month".format(sample_data_Mnth.index[0]))

In [ ]:
# Finding the probability of cancellation of flight for each carrier from the data
#import scipy. probability of cancellation is 0.02,
def prob(x):
  #r=np.sum(x==1)
  return np.sum(x==1)/np.size(x)
sample_data= main_df.sample(frac=0.6,replace=False)['Cancelled'].copy()
cancel_prob =sample_data.value_counts()/sample_data.shape[0]
print(cancel_prob[1])

In [ ]:
sample_data.value_counts()/sample_data.shape[0]

In [9]:
# Planes delay is same for older and newer planes
# Alternate hypothesis is older planes do suffer more delay
# confidence level is 95%,so alpha is 0.05%
planes_df1=planes_df[planes_df['type'].isnull()==False]
main_df_planes= main_df.merge(planes_df1[['tailnum','year']],how='left',left_on='TailNum',right_on='tailnum')
from scipy.stats import chisquare
main_df_planes['Delayed']= np.where((main_df_planes['DepDelay']!=0) |(main_df_planes['ArrDelay']!=0),1,0)
chi,p =chisquare(np.array(main_df_planes.groupby(['year'])['Delayed'].sum()))
print("P value {:.6f}".format(p))
if p <0.05:
  print("There sufficient evidence to regect null hypothes")
else:
  print("There  no sufficient evidence to reject null hypothes")
#Since null hypothesis is rejected, we can say older planes suffer more delay.

P value 0.000000
There sufficient evidence to regect null hypothes


* Can you detect cascading failures as delays in one airport create delays in others? Are there critical links in the system?
